In [16]:
import sqlite3

In [17]:
connection = sqlite3.connect('data/l1000.db')
cursor = connection.cursor()

In [18]:
command = '''
CREATE TABLE perts
(
    pert_id TEXT PRIMARY KEY NOT NULL,
    pert_iname TEXT NOT NULL,
    pert_type TEXT NOT NULL,
    num_gold INTEGER NOT NULL,
    num_inst INTEGER NOT NULL,
    num_sig INTEGER NOT NULL,
    in_summly INTEGER NOT NULL,
    inchi TEXT,
    inchi_key TEXT,
    pubchem_cid INTEGER
);
'''
cursor.execute(command);

In [19]:
#connection.commit()
connection.close()